In [1]:
import numpy as np
import pandas as pd
import os,time,re
import pickle,gzip

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
color=sns.color_palette()
import matplotlib as mpl
%matplotlib inline

In [3]:
#data prep and model evaluation
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [4]:
import lightgbm as lgb

In [5]:
import tensorflow as tf

2022-11-01 15:35:25.663664: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [6]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras import regularizers
from keras.losses import mse, binary_crossentropy

In [7]:
pwd

'/home/tfujiwara/important_data'

In [8]:
data=pd.read_csv("creditcard.csv")

In [9]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
dataX=data.copy().drop(['Class', 'Time'],axis=1)

In [11]:
dataY=data['Class'].copy()

In [12]:
featuresToScale=dataX.columns

In [13]:
featuresToScale

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')

In [14]:
sX=pp.StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
dataX.loc[:,featuresToScale]=sX.fit_transform(dataX[featuresToScale])

In [16]:
dataX.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,0.083386,...,0.326118,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,0.244964
1,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,-0.153350,...,-0.089611,-0.307377,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,-0.342475
2,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,0.190700,...,0.680975,0.337632,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,1.160686
3,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,-0.050468,...,-0.269855,-0.147443,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,0.140534
4,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,0.691625,...,0.529939,-0.012839,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,-0.073403


In [17]:
X_train, X_test, y_train, y_test=train_test_split(dataX,dataY, test_size=0.33, random_state=2022, stratify=dataY)

In [18]:
X_train_AE=X_train.copy()
X_test_AE=X_test.copy()

In [19]:
#anomalyScores calculation
def anomalyScores(originalDF,reducedDF):
    loss=np.sum((np.array(originalDF)-np.array(reducedDF))**2, axis=1)
    loss=pd.Series(data=loss,index=originalDF.index)
    loss=(loss-np.min(loss))/np.max(loss)-np.min(loss)
    return loss

In [20]:
#result visualization
def plotResults(trueLabels, anomalyScores, returnPreds=False):
    preds=pd.concat([trueLabels, anomalyScores],axis=1)
    preds.columns=['trueLabel', 'anomalyScore']
    precision, recall, thresholds = precision_recall_curve(preds['trueLabel'], preds['anomalyScore'])
    average_precision=average_precision_score(preds['trueLabel'],preds['anomalyScore'])
    plt.step(recall,precision,color='k', alpha=0.7,where='post')
    plt.fill_between(recall,precision,step='post',alpha=0.3, color='k')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0,1.05])
    plt.xlim([0.0,1.0])
    plt.title('Precision-Rcall curve: Average Precision={0:0.2f}'.format(average_precision))
    fpr,tpr,thresholds=roc_curve(preds['trueLabel'],preds['anomalyScore'])
    areaUnderROC=auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr,tpr,color='r',lw=2,label='ROC curve')
    plt.plot([0,1],[0,1],color='k',lw=2,linestyle='--')
    plt.xlim([0.0,1.0])
    plt.ylim([0.0,1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic: Area under the curve = {0:0.2f}'.format(areaUnderROC))
    plt.legend(loc="lower right")
    plt.show()
    
    if returnPreds==True:
        return preds

In [21]:
#preventing gpu's overflow
import tensorflow as tf
import tensorflow.keras.backend as tensorflow_backend

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

2022-11-01 15:35:30.428478: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 15:35:30.429606: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-01 15:35:30.430640: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-01 15:35:30.590925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:35:30.591293: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: Tesla

2022-11-01 15:35:31.201196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1261] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-11-01 15:35:31.201264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      0 1 
2022-11-01 15:35:31.201281: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1280] 0:   N N 
2022-11-01 15:35:31.201295: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1280] 1:   N N 
2022-11-01 15:35:31.201648: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:35:31.202304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:35:31.202782: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS h

In [22]:
#model creating
model=Sequential()
model.add(Dense(units=29, activation='linear', input_dim=29))
model.add(Dense(units=29,activation='linear'))
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
num_epochs=10
batch_size=4

history=model.fit(x=X_train_AE,y=X_train_AE,
                  epochs=num_epochs,
                  batch_size=batch_size,
                  shuffle=True,
                  validation_data=(X_train_AE,X_train_AE),
                  verbose=1)

2022-11-01 15:35:31.225682: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-01 15:35:31.226029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:35:31.226461: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: Tesla K20c computeCapability: 3.5
coreClock: 0.7055GHz coreCount: 13 deviceMemorySize: 4.63GiB deviceMemoryBandwidth: 193.71GiB/s
2022-11-01 15:35:31.226659: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:35:31.227084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:02:00.0 name: 

Epoch 1/10
    1/47705 [..............................] - ETA: 7:25:06 - loss: 1.4687 - accuracy: 0.0000e+00

2022-11-01 15:35:31.856103: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


47705/47705 [==============================] - 125s 3ms/step - loss: 0.1345 - accuracy: 0.8532 - val_loss: 2.5651e-04 - val_accuracy: 0.9979
Epoch 2/10
47705/47705 [==============================] - 124s 3ms/step - loss: 0.0030 - accuracy: 0.9841 - val_loss: 0.0029 - val_accuracy: 0.9733
Epoch 3/10
47705/47705 [==============================] - 124s 3ms/step - loss: 0.0026 - accuracy: 0.9854 - val_loss: 1.2696e-04 - val_accuracy: 0.9963
Epoch 4/10
47705/47705 [==============================] - 124s 3ms/step - loss: 0.0026 - accuracy: 0.9850 - val_loss: 6.9280e-05 - val_accuracy: 0.9971
Epoch 5/10
47705/47705 [==============================] - 125s 3ms/step - loss: 0.0030 - accuracy: 0.9845 - val_loss: 2.0820e-04 - val_accuracy: 0.9943
Epoch 6/10
47705/47705 [==============================] - 125s 3ms/step - loss: 0.0028 - accuracy: 0.9843 - val_loss: 0.0060 - val_accuracy: 0.9683
Epoch 7/10
47705/47705 [==============================] - 125s 3ms/step - loss: 0.0032 - accuracy: 0.9851 -

In [23]:
#model prediction
predictions=model.predict(X_test,verbose=1)
anomalyScoresAE=anomalyScores(X_test,predictions)
preds=plotResults(y_test,anomalyScoresAE, True)

2032/2938 [===================>..........] - ETA: 0s

2022-11-01 15:56:21.031760: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 8.00M (8388608 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:21.038490: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 7.20M (7549952 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:21.042859: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 6.48M (6795008 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:21.048317: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 5.83M (6115584 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:21.055927: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 5.25M (5504256 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:21.062278: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 4.72M (4953856 byte

2938/2938 [==============================] - 2s 784us/step


2022-11-01 15:56:21.887859: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 16.00M (16777216 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:21.892211: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 16.00M (16777216 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:31.896524: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 16.00M (16777216 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:31.900781: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 16.00M (16777216 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-01 15:56:31.900824: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 10.40MiB (rounded to 10902528)requested by op ConcatV2
Current allocation summary follows.
2022-11-01 15:56:31.900885: I tensorflow/core/common_ru

ResourceExhaustedError: OOM when allocating tensor with shape[93987,29] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ConcatV2] name: concat